In [1]:
from libcst import parse_module, Module, Expr, Pass, Comment, CSTTransformer, Comparison, CSTNode, ComparisonTarget
from libcst import matchers
from typing import  Union, Tuple, Type, Dict 
import re
from libcst import Equal, GreaterThanEqual
from libcst import matchers as m


## REFERENCES

- [Nodes](https://libcst.readthedocs.io/en/latest/nodes.html#libcst-nodes)

In [2]:
# Use difflib to show the changes to verify type annotations were added as expected.
import difflib

def printdiff(original_node, updated_node):
    return (
        "".join(
            difflib.unified_diff(original_node.splitlines(1), updated_node.splitlines(1))
        )
    )

In [48]:
import libcst as cst
from typing import Dict

def gen_two_transfomers(op0, op1, op2):
    print(op0, op1, op2)
    
    ## need to write a new provider IsModifiedProvider which checks if a node has already been tainted by a transformer when multiple transformers are called in a row
    ## all the comparisonprorivedrs should inherit from the IsModifiedProvider such that they can check if a node has already been modified by another transformer
    ## the IsModifiedProvider should be a BatchableMetadataProvider and 
    # should be automatically added to the metadatadependencies of the transformers because it is a base class for all the comparison providers
    # whenever a comparisonprovider succesfuly taints a node to be modified by the transformer it should use the parent class to set the metadata that can be used by the other comparison providers to check if the node has already been modified
    # this must be done because the comparison providers are called in a sequence and might overwrite each other's changes,
    #  also they can only access metadata of the same type so we use the shared base class to manage the communication between the comparison providers without using a global variable

    #let's start with a modified comparisonprovider which checks if a node has already been modified by another transformer using the IsModifiedProvider base class
    class IsModifiedProvider(cst.BatchableMetadataProvider[Dict]):
        def visit_ComparisonTarget(self, node: cst.ComparisonTarget) -> None:
            # Mark the node as an equal comparison node
            print("modified provider")
            self.set_metadata(node, dict(modified=False))

    class IsComparisonProviderFromModifiedProvider1(cst.BatchableMetadataProvider[Dict]):
        """A comparison provider which uses the IsModifiedProvider base class to check if a node has already been modified by another transformer"""
        METADATA_DEPENDENCIES = (IsModifiedProvider, )
        MATCHER = m.call_if_inside(m.ComparisonTarget(operator=[getattr(m, op0.__name__)()]))

        def visit_ComparisonTarget(self, node: cst.ComparisonTarget) -> None:
            # Check if the node matches the matcher
            if not self.MATCHER(): return
            # Check if the node has already been modified by another transformer by explicitly invoking the parent IsModifiedProvider get_metadata method
            # print("prima del modifided get 1 ")
            if self.get_metadata(IsModifiedProvider,node)['modified']: 
                print("mi fermo 1 ") 
                return
            # print("dopo del modifided get 1")
            # Mark the node as a target comparison node
            self.set_metadata(node, dict(comparison=node, 
                                    original=node.operator.__class__,
                                    modified=True,author=self.__class__.__name__))                
            # use the IsModifiedProvider to set the metadata that can be used by the other comparison providers to check if the node has already been modified
            # IsModifiedProvider.set_metadata(self,node, dict(modified=True, author=self.__class__.__name__))    

    class IsComparisonProviderFromModifiedProvider2(cst.BatchableMetadataProvider[Dict]):
        """A comparison provider which uses the IsModifiedProvider base class to check if a node has already been modified by another transformer"""
        METADATA_DEPENDENCIES = (IsModifiedProvider, )
        MATCHER = m.call_if_inside(m.ComparisonTarget(operator=[getattr(m, op1.__name__)()]))

        def visit_ComparisonTarget(self, node: cst.ComparisonTarget) -> None:
            # Check if the node matches the matcher
            if not self.MATCHER(): return
            # Check if the node has already been modified by another transformer by explicitly invoking the parent IsModifiedProvider get_metadata method
            print("prima del modifided get 2")
            if self.get_metadata(IsModifiedProvider,node)['modified']: 
                print("mi fermo 2") 
                return
            print("dopo del modifided get 2")
            # Mark the node as a target comparison node
            self.set_metadata(node, dict(comparison=node, 
                                    original=node.operator.__class__,
                                    modified=True,author=self.__class__.__name__))                
            # use the IsModifiedProvider to set the metadata that can be used by the other comparison providers to check if the node has already been modified
            # IsModifiedProvider.set_metadata(self,node, dict(modified=True, author=self.__class__.__name__))
    
    class ApplyTransformer1(cst.CSTTransformer):
        METADATA_DEPENDENCIES = (IsComparisonProviderFromModifiedProvider1, )
    
        def leave_ComparisonTarget(self, original_node:cst.ComparisonTarget, updated_node: cst.ComparisonTarget) -> None:
            meta = self.get_metadata(IsComparisonProviderFromModifiedProvider1, original_node)
            #check if the author of the metadata is a provider of the same type of the one in metadata_dependencies
            # print(meta)
            if meta['author'] in [x.__name__ for x in self.METADATA_DEPENDENCIES]:
                # print("sono qui  1")
                updated_node = meta['comparison'].with_changes(operator=op1()) # OP2
            return updated_node

        def __repr__(self):
            return super().__repr__(self) + ':' + op0.__name__ +':' + op1.__name__

    
    class ApplyTransformer2(cst.CSTTransformer):
        METADATA_DEPENDENCIES = (IsComparisonProviderFromModifiedProvider2, )
    
        def leave_ComparisonTarget(self, original_node:cst.ComparisonTarget, updated_node: cst.ComparisonTarget) -> None:
            meta = self.get_metadata(IsComparisonProviderFromModifiedProvider2, original_node)
            #check if the author of the metadata is a provider of the same type of the one in metadata_dependencies
            print(meta)
            if meta['author'] in [x.__name__ for x in self.METADATA_DEPENDENCIES]:
                print("sono qui 2")
                updated_node = meta['comparison'].with_changes(operator=op2())



    return ApplyTransformer1 , ApplyTransformer2     



In [49]:
# Test
from libcst import (Equal, GreaterThan, GreaterThanEqual, LessThan,
                    LessThanEqual, NotEqual, NotIn, In, Is, IsNot, Not, And, Or, Match)
import collections

str2op = dict([
    ('==', Equal),
    ('>', GreaterThan),
    ('<=', LessThanEqual),
    ('>=', GreaterThanEqual),
    ('<', LessThan),
    ('!=', NotEqual),
    ('not in', NotIn),
    ('in', In),
    ('is', Is),
    ('is not', IsNot),
    ('not', Not),
    ('and', And),
    ('or', Or),
    ('match', Match),
])

op2str = {v:k for k,v in str2op.items()}

# Get the script as a string
script = "x != 1 + 2 == 3"

# Parse the script into a CST
module = cst.parse_module(script)

# Use the gen_two_transfomers function to generate the ApplyTransformer1 and ApplyTransformer2 classes
ApplyTransformer1, ApplyTransformer2 = gen_two_transfomers(str2op['!='], str2op['=='], str2op['<='])

wrapper = cst.MetadataWrapper(module)
# Apply the ApplyTransformer1 to the CST
updated_node = wrapper.visit(ApplyTransformer1())
# Apply the ApplyTransformer2 to the CST
updated_node = wrapper.visit(ApplyTransformer2())

print(updated_node.code)


modified provider
modified provider
prima del modifided get 2
dopo del modifided get 2
prima del modifided get 2
dopo del modifided get 2
{'comparison': ComparisonTarget(
    operator=NotEqual(
        value='!=',
        whitespace_before=SimpleWhitespace(
            value=' ',
        ),
        whitespace_after=SimpleWhitespace(
            value=' ',
        ),
    ),
    comparator=BinaryOperation(
        left=Integer(
            value='1',
            lpar=[],
            rpar=[],
        ),
        operator=Add(
            whitespace_before=SimpleWhitespace(
                value=' ',
            ),
            whitespace_after=SimpleWhitespace(
                value=' ',
            ),
        ),
        right=Integer(
            value='2',
            lpar=[],
            rpar=[],
        ),
        lpar=[],
        rpar=[],
    ),
), 'original': <class 'libcst._nodes.op.NotEqual'>, 'modified': True, 'author': 'IsComparisonProviderFromModifiedProvider2'}
sono qui 2


Exception: Expected a node of type CSTNode or a RemovalSentinel, but got a return value of NoneType

In [ ]:
def mutation_factory(ops_list):
    forward_transformers = []
    backward_transformers = []
    class ModifiedNodeProvider(cst.BatchableMetadataProvider[Dict]):
        def __init__(self):
            self._modified_nodes = set()

        def visit_Comparison(self, node: cst.Comparison) -> None:
            self._modified_nodes.add(node)
            
        def is_modified(self, node: cst.Comparison) -> None:
            return node in self._modified_nodes
    
    for op0, op2 in ops_list:
        class IsComparisonProvider(cst.BatchableMetadataProvider[Dict]):
            MATCHER = m.call_if_inside(m.ComparisonTarget(operator=[getattr(m, op0.__name__)()]))
            def visit_Comparison(self, node: cst.Comparison) -> None:
                if not self.MATCHER(): return
                self.set_metadata(node, dict(comparison=node.comparisons[0], 
                                            original=node.comparisons[0].operator.__class__))
                
        class ApplyTransformer(cst.CSTTransformer):
            METADATA_DEPENDENCIES = (IsComparisonProvider, ModifiedNodeProvider)
            def leave_Comparison(self, node: cst.Comparison) -> None:
                meta = self.get_metadata(IsComparisonProvider, node)
                if meta:
                    if self.get_metadata(ModifiedNodeProvider, node).is_modified(node):
                        return node
                    updated_comparison = meta['comparison'].with_changes(operator=op2()) 
                    node = node.with_changes(comparisons=[updated_comparison])
                return node
        
        class ReverseTransformer(cst.CSTTransformer):
            METADATA_DEPENDENCIES = (IsComparisonProvider, ModifiedNodeProvider)

            def leave_Comparison(self, node: cst.Comparison) -> cst.Comparison:
                meta = self.get_metadata(IsComparisonProvider, node)
                if meta:
                    modified_meta = self.get_metadata(ModifiedNodeProvider, node)
                    if modified_meta and modified_meta.get('modified'):
                        updated_comparison = meta['comparison'].with_changes(operator=meta['original']())
                        node = node.with_changes(comparisons=[updated_comparison])
                        modified_meta['modified'] = False
                return node
        forward_transformers.append(ApplyTransformer)
        backward_transformers.append(ReverseTransformer)
    return forward_transformers, backward_transformers

           


In [ ]:
from libcst import (Equal, GreaterThan)

# Define the list of tuple operations
ops_list = [(str2op['=='], str2op['>'])]

# Generate the forward and backward transformers
forward_transformers, backward_transformers = mutation_factory(ops_list)

# Get the script as a string
script = "x >= 1 + 2 == 3"

# Parse the script into a CST
module = cst.parse_module(script)

# Use the MetadataWrapper to apply the forward transformers
wrapper = cst.MetadataWrapper(module)
for transformer in forward_transformers:
    wrapper.visit(transformer())

# Print the transformed code
print(wrapper.module.code)

# Use the MetadataWrapper to apply the backward transformers
for transformer in backward_transformers:
    wrapper.visit(transformer())

# Print the original code
print(wrapper.module.code)


AttributeError: 'ModifiedNodeProvider' object has no attribute '_computed'